In [1]:
from landsatxplore.api import API
from glob import glob
import os, sys

In [2]:
# Initialize a new API instance and get an access key
api = API("pinnacle55", "KM89xq6rSRZ6zJP")

In [13]:
# list of years to search
years = range(2023, 2024)

scene_dict = {}

for year in years:

    # Search for Landsat TM scenes
    scenes = api.search(
        dataset='landsat_ot_c2_l2',
        latitude=32.754418985494425,
        longitude=129.8764305203059,
        start_date=f'{year}-05-01', # rainy season ends in Jan, this should be mostly cloudless
        end_date=f'{year+1}-06-01',
        max_cloud_cover=5
    )
    
    # sort scenes by amount of cloud cover
    scenes = sorted(scenes, key = lambda x: x["cloud_cover"])
    
    # store scenes in dictionary
    scene_dict[str(year)] = scenes

In [15]:
len(scene_dict['2023'])

2

In [15]:
# check how many scenes per year
# to avoid over downloading (each scene is ~1GB, set limit to 5 scenes per year)
for key in scene_dict:
    print(key, len(scene_dict[key]))

2013 2
2014 7
2015 6
2016 8
2017 6
2018 5
2019 6
2020 4
2021 5
2022 6


In [16]:
from landsatxplore.earthexplorer import EarthExplorer

# Sign in to earthexplorer
ee = EarthExplorer("pinnacle55", "KM89xq6rSRZ6zJP")

for key in scene_dict:
    
    print(f"Downloading scenes in {key}.")

    scene_count = 0
    
    for scene in scene_dict[key]:     
        
        # Limit downloads to 5 scenes
        # Scenes are sorted from lowest to highest cloud_cover, so limiting the scenes should be alright
        if scene_count >= 5:
            break
        
        print(f"Attempting to download {scene['display_id']}")
        
        
        # note that the appears to be an issue where the program will claim that the download failed but 
        # it actually succeeded - despite this the program will throw an error and the script will not
        # continue. As a workaround use a try except block to continue the loop despite the error being 
        # thrown.
        try:
            ee.download(scene['display_id'], output_dir = os.getcwd())
        except:
            print(f"{scene['display_id']} may have failed to download! Check to be sure.")
            continue
        
        # if the file was downloaded successfully, add to scene_count
        
        if os.path.exists(f"{scene['display_id']}.tar"):
            print(f"{scene['display_id']} downloaded successfully!")
            scene_count += 1

ee.logout()

Attempting to download LC09_L2SP_113037_20230510_20230512_02_T1
Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 772M/772M [05:49<00:00, 2.32MB/s]


LC09_L2SP_113037_20230510_20230512_02_T1 downloaded successfully!
Attempting to download LC08_L2SP_113037_20230603_20230607_02_T1
Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 821M/821M [05:51<00:00, 2.45MB/s]


LC08_L2SP_113037_20230603_20230607_02_T1 downloaded successfully!


In [16]:
### Code to set up a bunch of folders in for data download
# base_dir = "./data_collated"

# years = range(2013, 2023)

# for year in years:
#     year_folder = os.path.join(base_dir, str(year))
#     if not os.path.exists(year_folder):
#         os.mkdir(year_folder)

In [8]:
existing_scenes = os.listdir("./data")

scenes_to_dl = []

for scene in scenes:
    # be wary - for some reason, landsat_product_id gives the L1TP products
    # but display_id gives the L2SP products
    # choose the product you want carefully
    product_id = scene['display_id']
    if product_id not in existing_scenes:
        scenes_to_dl.append(product_id)

LC08_L2SP_125059_20210703_20210712_02_T1
LC08_L2SP_125059_20210601_20210608_02_T1
LC08_L2SP_125059_20210313_20210318_02_T1


In [3]:
from landsatxplore.earthexplorer import EarthExplorer

# Sign in to earthexplorer
ee = EarthExplorer("pinnacle55", "KM89xq6rSRZ6zJP")

for scene in scenes_to_dl:
    # note that the appears to be an issue where the program will claim that the download failed but 
    # it actually succeeded - despite this the program will throw an error and the script will not
    # continue. As a workaround use a try except block to continue the loop despite the error being 
    # thrown.

    try:
        ee.download(scene, output_dir='./data_cloud_imputing')
    except:
        print(f"{scene} may have failed to download! Check to be sure.")
        continue

ee.logout()

4

In [1]:
### older attempt using a combination of command line and pythonic env ###
# prior to executing this code I ran the following in the command line colon 
# landsatxplore search --dataset landsat_ot_c2_l2 --location 1.3521 103.8198 --start 2013-01-01 --end 2023-01-01 --clouds 20 > results.csv
# this allows me to find all scenes involving Singapore that have a cloud coverage of 20% or less;
# these results are then saved into a CSV file
# I then manually selected all scenes from the same month in order to remove all seasonal biases

from landsatxplore.earthexplorer import EarthExplorer

# Sign in to earthexplorer
ee = EarthExplorer("username", "password")

# Read the aforementioned csv and get the scene IDs
with open("results.csv") as f:
    lines = [line.rstrip('\n') for line in f]
f.close()

for scene in lines:
    # note that the appears to be an issue where the program will claim that the download failed but 
    # it actually succeeded - despite this the program will throw an error and the script will not
    # continue. As a workaround use a try except block to continue the loop despite the error being 
    # thrown.

    try:
        ee.download(scene, output_dir='./data_cloud_imputing')
    except:
        print(f"{scene} may have failed to download! Check to be sure.")
        continue

ee.logout()

Download failed with dataset id 1 of 3. Re-trying with the next one.
Download failed with dataset id 2 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 809M/809M [04:29<00:00, 3.14MB/s]


Download failed with dataset id 1 of 3. Re-trying with the next one.


810MB [01:58, 7.14MB/s]                                                                                                


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20220519_20220525_02_T1 may have failed to download! Check to be sure.
Download failed with dataset id 1 of 3. Re-trying with the next one.


787MB [01:55, 7.14MB/s]                                                                                                


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20220417_20220420_02_T1 may have failed to download! Check to be sure.
Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 830M/830M [02:08<00:00, 6.76MB/s]


None of the archived ids succeeded! Update necessary!
LC09_L2SP_125059_20220324_20230424_02_T1 may have failed to download! Check to be sure.


In [5]:
for scene in lines:
    # note that the appears to be an issue where the program will claim that the download failed but 
    # it actually succeeded - despite this the program will throw an error and the script will not
    # continue. As a workaround use a try except block to continue the loop despite the error being 
    # thrown.

    try:
        ee.download(scene, output_dir='./data')
    except:
        print(f"{scene} failed to download!")
        continue

ee.logout()

Download failed with dataset id 1 of 3. Re-trying with the next one.
None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20220401_20220406_02_T1 failed to download!
Download failed with dataset id 1 of 3. Re-trying with the next one.


773MB [03:01, 4.46MB/s]                                                                                                


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20210617_20210622_02_T1 failed to download!
Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 748M/748M [02:48<00:00, 4.64MB/s]


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20200716_20200911_02_T1 failed to download!
Download failed with dataset id 1 of 3. Re-trying with the next one.


...

LC08_L2SP_125059_20160502_20200907_02_T1 failed to download!
LC08_L2SP_125059_20150414_20200909_02_T1 failed to download!
Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 743M/743M [03:01<00:00, 4.28MB/s]


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20140513_20200911_02_T1 failed to download!
Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|███████████████████████████████████████████████████████████████████████████████| 671M/671M [02:51<00:00, 4.11MB/s]


None of the archived ids succeeded! Update necessary!
LC08_L2SP_125059_20130627_20200912_02_T1 failed to download!
